# Marking up the residential registry with updates from **Public data**

# Install dependencies

In [11]:
!pip install git+https://github.com/Valueguard-Index-Sweden/valueguard-python-client#egg=valueguard
!pip install tqdm

    100% |████████████████████████████████| 81kB 11.0MB/s 


## Import libraries 

- **valueguard** our official client to retrive data from the API
- **tqdm** for a easily understandable progressbar
- **pandas** to work with the data in python
- **math** to calcualate the ceiling of pages needed
- **time** to time the functions

In [14]:
import valueguard
from tqdm import tqdm
import pandas as pd
import math
import time
import datetime

## Display Settings

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
vgClient = valueguard.Client()

## Get credentials 

In [59]:
valueguard_username = os.getenv('VALUEGUARD_API_USERNAME')
valueguard_password = os.getenv('VALUEGUARD_API_PASSWORD')

## Login to Valueguard

In [51]:
vgClient.authenticate(valueguard_username,valueguard_password)

In [9]:
residential_registry_df = pd.read_csv("../data/residential_registry.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
residential_registry_df.set_index('id', inplace=True) # Set the id in the data to the id of the dataframe

In [56]:
# Get the max updated at value
residential_registry_updated_at_max = residential_registry_df['updated_at'].max()

## Settings for the **residential registry markups**

In [57]:
residential_registry_markups_search_criteria={
    'updated_at_min': residential_registry_updated_at_max,
    'updated_from':"public_data"
}

residential_registry_markups_total_nr_records = vgClient.residential_registry_markups(offset=0, limit=0, search_criteria=residential_registry_markups_search_criteria)['meta_data']['total_nr_records']
page_size = 50000

pages = math.ceil(residential_registry_markups_total_nr_records/page_size)

### New rows to update registry with

In [58]:
print(residential_registry_markups_total_nr_records)

32147


## Addring the markups to the **residential registry**

In [50]:
for page_nr in tqdm(range(pages)):
    for residenetial_registry_markup in vgClient.residential_registry_markups(offset=page_nr*page_size, limit=page_size, search_criteria=residential_registry_markups_search_criteria)['residences']:
        id_to_update = residenetial_registry_markup['id']
        for column_to_update in residenetial_registry_markup.keys():
            if column_to_update != "id":
                residential_registry_df.loc[int(id_to_update),column_to_update] = residenetial_registry_markup[column_to_update]

100%|██████████| 1/1 [8:26:11<00:00, 30371.95s/it]


### Save data

### File (CSV)

In [ ]:
residential_registry_df.to_csv("../data/residential_registry_with_updates.csv")

### MySql (Database)

In [ ]:
# Credentials to database connection
hostname="localhost"
dbname="mydb_name"
uname="my_user_name"
pwd="my_password"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

# Convert dataframe to sql table                                   
# --> residential_registry_df.to_sql('residential_registry_with_updates', engine, index=False)